In [1]:
#import all neccessary libraries
import pandas as pd
import numpy as np
import re
import string
import nltk
nltk.download("all")
nltk.download('wordnet')
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import OneHotEncoder
import textblob

from sklearn.feature_extraction.text import TfidfVectorizer
pd.set_option('max_colwidth', 100)


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\91951\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\91951\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\91951\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\91951\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     C:\Users\91951\AppData\Roaming\nltk_data...
[nltk_data]    | 

In [2]:
#import dataframe
data=pd.read_csv("sample.csv")

In [3]:
#check data
data.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,119237,105834,True,Wed Oct 11 06:55:44 +0000 2017,@AppleSupport causing the reply to be disregarded and the tapped notification under the keyboard...,119236,NaN
1,119238,ChaseSupport,False,Wed Oct 11 13:25:49 +0000 2017,"@105835 Your business means a lot to us. Please DM your name, zip code and additional details ab...",NaN,119239.0
2,119239,105835,True,Wed Oct 11 13:00:09 +0000 2017,@76328 I really hope you all change but I'm sure you won't! Because you don't have to!,119238,NaN
3,119240,VirginTrains,False,Tue Oct 10 15:16:08 +0000 2017,@105836 LiveChat is online at the moment - https://t.co/SY94VtU8Kq or contact 03331 031 031 opti...,119241,119242.0
4,119241,105836,True,Tue Oct 10 15:17:21 +0000 2017,@VirginTrains see attached error message. I've tried leaving a voicemail several times in the pa...,119243,119240.0


In [4]:
data["text"][0]

'@AppleSupport causing the reply to be disregarded and the tapped notification under the keyboard is opened😡😡😡'

In [5]:
data["text"]

0     @AppleSupport causing the reply to be disregarded and the tapped notification under the keyboard...
1     @105835 Your business means a lot to us. Please DM your name, zip code and additional details ab...
2                  @76328 I really hope you all change but I'm sure you won't! Because you don't have to!
3     @105836 LiveChat is online at the moment - https://t.co/SY94VtU8Kq or contact 03331 031 031 opti...
4     @VirginTrains see attached error message. I've tried leaving a voicemail several times in the pa...
                                                     ...                                                 
88    @105860 I wish Amazon had an option of where I can just get it shipped to the ups store so I can...
89                                           They reschedule my shit for tomorrow https://t.co/RsvZcT982t
90    @105861 Hey Sara, sorry to hear of the issues you are having, can I ask if it's the lay out or j...
91    @Tesco bit of both - finding the layout 

In [6]:
data["text"][10]

'@105837 We can help. Which version of iOS are you on? You can find that in Settings &gt; General &gt; About. Reply in DM. https://t.co/GDrqU22YpT'

In [7]:

# Function to replace multiple substrings
def replace_multiple(string, replacements):
    new_string = string
    for old_value, new_value in replacements.items():
        new_string = new_string.replace(old_value, new_value)
    return new_string

# Dictionary of replacements
replacements = {"&gt;": ">", "&lt;": "<", "&amp;": "&"}

# Apply the function to the DataFrame column
data['text'] = data['text'].apply(lambda x: replace_multiple(x, replacements))



In [8]:
data["text"][10]

'@105837 We can help. Which version of iOS are you on? You can find that in Settings > General > About. Reply in DM. https://t.co/GDrqU22YpT'

In [9]:
data.shape

(93, 7)

In [10]:
#remove url function
def remove_url(text):
    pattern=re.compile(r'https?://\S+|www\.\S+')
    return pattern.sub("",text)


data["text"]=data["text"].apply(remove_url)

In [11]:
data["text"][10]

'@105837 We can help. Which version of iOS are you on? You can find that in Settings > General > About. Reply in DM. '

In [12]:
data["text"][24]

'@AppleSupport after the 11.0.2 my phone just sucks most of the apps are broken, wifi disconnects frequently #apple #ios1102 #painfulupdate'

In [13]:
data["text"][9:45]

9              @105836 That's what we're here for Miriam 😊  The team should send you an email shortly ^HP
10    @105837 We can help. Which version of iOS are you on? You can find that in Settings > General > ...
11                               @105838 @AppleSupport Me too am suffering , hope the can find a solution
12    @AppleSupport hi #apple, I’ve a concern about the latest ios is too slow on #iphone6 and i am no...
13    @105839 Thanks for reaching out to us. We are always happy to help. Send us a DM so we can look ...
14    I just updated my phone and suddenly everything takes ages to load wtf @76099 this update sux I ...
15    @105840 Hi there! What device is this happening on? If you could also let us know the Android an...
16    @SpotifyCares Thanks! Version 8.4.22.857 armv7 on anker bluetooth speaker on Samsung Galaxy Tab ...
17    @105840 Thanks. The distance could possibly affect playback. Does logging out > restarting your ...
18    @SpotifyCares No, but I've moved speaker

In [14]:
# Function to remove hashtags
def remove_hashtags(sentence):
    words = sentence.split()
    without_hashtags = [word for word in words if not word.startswith('#')]
    return ' '.join(without_hashtags)

# Apply the function to the DataFrame column
data['text'] = data['text'].apply(lambda x: remove_hashtags(x))



In [15]:
data["text"][24]

'@AppleSupport after the 11.0.2 my phone just sucks most of the apps are broken, wifi disconnects frequently'

In [16]:
#remove punctuation
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [17]:
#collect in variable exclude
exclude=string.punctuation

In [18]:
#remove punctuation
def remove_punc(text):
    return text.translate(str.maketrans("","",exclude))

data["text"]=data["text"].apply(remove_punc)

In [19]:
data["text"][10]

'105837 We can help Which version of iOS are you on You can find that in Settings  General  About Reply in DM'

In [20]:
#remove Slang
chat_words={
   "DM":"Direct Message",
   "TY":"Thank You",
   "AC":"Account",
"AY": "Assessment Year",
"HP":"Happy",
"CP":"coupling",
"MM":"Message me",
"AD":"Advertisement",
"MU":"Miss You",
"RR":"Reply Requested",
   "JR":"Junior"

}

In [21]:
#remove slang and apply function
def chat_conversion(text):
    new_text=[]
    for w in text.split():
        if w.upper() in chat_words:
            new_text.append(chat_words[w.upper()])
        else:
            new_text.append(w)
    return " ".join(new_text)

In [22]:
#apply function
data["text"]=data["text"].apply(chat_conversion)

In [23]:
data["text"][10]

'105837 We can help Which version of iOS are you on You can find that in Settings General About Reply in Direct Message'

In [24]:
data["text"][9:45]

9                105836 Thats what were here for Miriam 😊 The team should send you an email shortly Happy
10    105837 We can help Which version of iOS are you on You can find that in Settings General About R...
11                                   105838 AppleSupport Me too am suffering hope the can find a solution
12    AppleSupport hi I’ve a concern about the latest ios is too slow on and i am not happy with it An...
13    105839 Thanks for reaching out to us We are always happy to help Send us a Direct Message so we ...
14    I just updated my phone and suddenly everything takes ages to load wtf 76099 this update sux I h...
15    105840 Hi there What device is this happening on If you could also let us know the Android and S...
16    SpotifyCares Thanks Version 8422857 armv7 on anker bluetooth speaker on Samsung Galaxy Tab A 201...
17    105840 Thanks The distance could possibly affect playback Does logging out restarting your devic...
18    SpotifyCares No but Ive moved speaker to

In [25]:
#remove Emoji
def remove_emojis_manually(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F700-\U0001F77F"  # alchemical symbols
                               u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
                               u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
                               u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                               u"\U0001FA00-\U0001FA6F"  # Chess Symbols
                               u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                               u"\U00002702-\U000027B0"  # Dingbats
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    clean_text = emoji_pattern.sub(r'', text)
    return clean_text

In [26]:
data["text"]=data["text"].apply(remove_emojis_manually)

In [27]:
data["text"][20]

'SpotifyCares Brilliant thanks '

In [28]:
#remove numbers from sentance
def remove_numbers(sentence):

    return re.sub(r'\d+', '', sentence)

# Apply the function to the DataFrame column
data["text"] = data["text"].apply(remove_numbers)

In [29]:
data["text"][0]

'AppleSupport causing the reply to be disregarded and the tapped notification under the keyboard is opened'

In [30]:
import nltk

from nltk.corpus import stopwords


In [31]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\91951\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [32]:
from nltk.tokenize import word_tokenize,sent_tokenize

In [33]:
stopwords.words("english")

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [34]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\91951\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [35]:
print(data["text"])

0     AppleSupport causing the reply to be disregarded and the tapped notification under the keyboard ...
1      Your business means a lot to us Please Direct Message your name zip code and additional details...
2                              I really hope you all change but Im sure you wont Because you dont have to
3         LiveChat is online at the moment or contact    option    Leave a message to request a call back
4     VirginTrains see attached error message Ive tried leaving a voicemail several times in the past ...
                                                     ...                                                 
88     I wish Amazon had an option of where I can just get it shipped to the ups store so I can avoid ...
89                                                                   They reschedule my shit for tomorrow
90     Hey Sara sorry to hear of the issues you are having can I ask if its the lay out or just the sp...
91    Tesco bit of both finding the layout cum

In [36]:
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()

# add stemming and lemmatisation in the preprocess function
def preprocess(document):
    'changes document to lower case and removes stopwords'

    # change sentence to lower case
    document = document.lower()

    # tokenize into words
    words = word_tokenize(document)

    # remove stop words
    words = [word for word in words if word not in stopwords.words("english")]

    # stem
    #words = [stemmer.stem(word) for word in words]

    # join words to make sentence
    document = " ".join(words)

    return document



In [37]:
# extract the messages from the dataframe
messages = [message for message in data["text"]]
print(messages)

['AppleSupport causing the reply to be disregarded and the tapped notification under the keyboard is opened', ' Your business means a lot to us Please Direct Message your name zip code and additional details about your concern Reply Requested', ' I really hope you all change but Im sure you wont Because you dont have to', ' LiveChat is online at the moment or contact    option    Leave a message to request a call back', 'VirginTrains see attached error message Ive tried leaving a voicemail several times in the past week', ' Have you tried from another device Miriam Message me', 'VirginTrains yep Ive tried laptop too several times over the past week and again today Ive tried different browsers too', ' Its working OK from here Miriam Does this link help Message me', 'VirginTrains I still havent heard the number Im directed to by phone is a dead end the live chat doesnt work Can someone call me', ' Thats what were here for Miriam  The team should send you an email shortly Happy', ' We can

In [38]:
documents = [preprocess(document) for document in messages]
print(documents)

['applesupport causing reply disregarded tapped notification keyboard opened', 'business means lot us please direct message name zip code additional details concern reply requested', 'really hope change im sure wont dont', 'livechat online moment contact option leave message request call back', 'virgintrains see attached error message ive tried leaving voicemail several times past week', 'tried another device miriam message', 'virgintrains yep ive tried laptop several times past week today ive tried different browsers', 'working ok miriam link help message', 'virgintrains still havent heard number im directed phone dead end live chat doesnt work someone call', 'thats miriam team send email shortly happy', 'help version ios find settings general reply direct message', 'applesupport suffering hope find solution', 'applesupport hi ’ concern latest ios slow happy solution please', 'thanks reaching us always happy help send us direct message look together', 'updated phone suddenly everythin

In [39]:
#Creating bag of words model using count vectorizer function

vectorizer = CountVectorizer()
bow_model = vectorizer.fit_transform(documents)
print(bow_model)

  (0, 27)	1
  (0, 72)	1
  (0, 356)	1
  (0, 125)	1
  (0, 434)	1
  (0, 294)	1
  (0, 229)	1
  (0, 304)	1
  (1, 356)	1
  (1, 58)	1
  (1, 269)	1
  (1, 260)	1
  (1, 473)	1
  (1, 323)	1
  (1, 121)	1
  (1, 270)	1
  (1, 288)	1
  (1, 509)	1
  (1, 84)	1
  (1, 1)	1
  (1, 117)	1
  (1, 91)	1
  (1, 358)	1
  (2, 344)	1
  (2, 197)	1
  :	:
  (91, 101)	1
  (91, 354)	1
  (91, 219)	1
  (91, 146)	1
  (91, 202)	1
  (91, 393)	1
  (91, 227)	1
  (91, 448)	1
  (91, 176)	1
  (91, 247)	1
  (92, 323)	1
  (92, 121)	1
  (92, 270)	1
  (92, 288)	1
  (92, 118)	1
  (92, 189)	1
  (92, 127)	1
  (92, 134)	1
  (92, 442)	1
  (92, 163)	1
  (92, 2)	1
  (92, 209)	1
  (92, 55)	1
  (92, 254)	1
  (92, 242)	1


In [40]:
#vector vocabulary
vectorizer.vocabulary_

{'applesupport': 27,
 'causing': 72,
 'reply': 356,
 'disregarded': 125,
 'tapped': 434,
 'notification': 294,
 'keyboard': 229,
 'opened': 304,
 'business': 58,
 'means': 269,
 'lot': 260,
 'us': 473,
 'please': 323,
 'direct': 121,
 'message': 270,
 'name': 288,
 'zip': 509,
 'code': 84,
 'additional': 1,
 'details': 117,
 'concern': 91,
 'requested': 358,
 'really': 344,
 'hope': 197,
 'change': 74,
 'im': 206,
 'sure': 425,
 'wont': 498,
 'dont': 129,
 'livechat': 251,
 'online': 302,
 'moment': 283,
 'contact': 94,
 'option': 307,
 'leave': 240,
 'request': 357,
 'call': 64,
 'back': 41,
 'virgintrains': 479,
 'see': 370,
 'attached': 37,
 'error': 138,
 'ive': 220,
 'tried': 459,
 'leaving': 241,
 'voicemail': 481,
 'several': 378,
 'times': 453,
 'past': 316,
 'week': 487,
 'another': 22,
 'device': 118,
 'miriam': 278,
 'yep': 505,
 'laptop': 234,
 'today': 454,
 'different': 120,
 'browsers': 56,
 'working': 500,
 'ok': 298,
 'link': 248,
 'help': 189,
 'still': 415,
 'havent'

In [41]:
# print the full sparse matrix
print(bow_model.toarray())

[[0 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 1]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]]


In [42]:
print(bow_model.shape)
print(vectorizer.get_feature_names_out())

(93, 510)
['account' 'additional' 'address' 'adnams' 'advertisement' 'advise'
 'affect' 'age' 'ages' 'ago' 'agoura' 'air' 'aircraft' 'alcohol' 'allow'
 'almost' 'also' 'always' 'amanda' 'amazon' 'android' 'anker' 'another'
 'answer' 'anything' 'app' 'apple' 'applesupport' 'appreciate' 'apps'
 'armv' 'article' 'ask' 'askspectrum' 'ass' 'assessment' 'assistance'
 'attached' 'avoid' 'away' 'ba' 'back' 'basket' 'battery' 'beatner' 'beg'
 'behaviors' 'believes' 'bit' 'blind' 'bluetooth' 'brilliant'
 'britishairways' 'broadside' 'broken' 'browser' 'browsers' 'bug'
 'business' 'buying' 'bye' 'cable' 'cache' 'cachecookies' 'call' 'called'
 'calling' 'calls' 'cares' 'carry' 'cause' 'caused' 'causing' 'challenged'
 'change' 'charge' 'chat' 'check' 'cheers' 'choice' 'chris' 'click'
 'close' 'coaxial' 'code' 'colleague' 'colleagues' 'come' 'comes'
 'commonsense' 'companion' 'concern' 'congrats' 'constantly' 'contact'
 'cookies' 'correct' 'could' 'coupling' 'crash' 'crossed' 'cumbersome'
 'curiousw

Stemming and lemmatising

In [43]:


stemmer = PorterStemmer()
wordnet_lemmatizer = WordNetLemmatizer()

# add stemming and lemmatisation in the preprocess function
def preprocess(document, stem=True):
    'changes document to lower case and removes stopwords'

    # change sentence to lower case
    document = document.lower()

    # tokenize into words
    words = word_tokenize(document)

    # remove stop words
    words = [word for word in words if word not in stopwords.words("english")]

    if stem:
        words = [stemmer.stem(word) for word in words]
    else:
        words = [wordnet_lemmatizer.lemmatize(word, pos='v') for word in words]

    # join words to make sentence
    document = " ".join(words)

    return document

Bag of words model on stemmed messages

In [44]:
# stem messages
stem_messages = [preprocess(message, stem=True) for message in  documents]

# bag of words model
vectorizer = CountVectorizer()
bow_model = vectorizer.fit_transform(stem_messages)

In [45]:
# look at the dataframe
pd.DataFrame(bow_model.toarray(), columns = vectorizer.get_feature_names_out())

,account,addit,address,adnam,advertis,advis,affect,age,ago,agoura,...,work,wtf,xfiniti,yard,year,yep,youll,your,youv,zip
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
89,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
90,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
91,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [46]:
print(vectorizer.get_feature_names_out())

['account' 'addit' 'address' 'adnam' 'advertis' 'advis' 'affect' 'age'
 'ago' 'agoura' 'air' 'aircraft' 'alcohol' 'allow' 'almost' 'also' 'alway'
 'amanda' 'amazon' 'android' 'anker' 'anoth' 'answer' 'anyth' 'app' 'appl'
 'applesupport' 'appreci' 'armv' 'articl' 'ask' 'askspectrum' 'ass'
 'assess' 'assist' 'attach' 'avoid' 'away' 'ba' 'back' 'basket' 'batteri'
 'beatner' 'beg' 'behavior' 'believ' 'bit' 'blind' 'bluetooth' 'brilliant'
 'britishairway' 'broadsid' 'broken' 'browser' 'bug' 'busi' 'buy' 'bye'
 'cabl' 'cach' 'cachecooki' 'call' 'care' 'carri' 'caus' 'challeng'
 'chang' 'charg' 'chat' 'check' 'cheer' 'choic' 'chri' 'click' 'close'
 'coaxial' 'code' 'colleagu' 'come' 'commonsens' 'companion' 'concern'
 'congrat' 'constantli' 'contact' 'cooki' 'correct' 'could' 'coupl'
 'crash' 'cross' 'cumbersom' 'curiouswil' 'current' 'custom' 'dani' 'day'
 'dead' 'delay' 'delet' 'deliv' 'deliveri' 'depart' 'departur' 'detail'
 'devic' 'differ' 'direct' 'disconnect' 'disgrac' 'disregard' 'dis

Now Apply lemmatizing the messages.

In [47]:
# lemmatise messages
lemma_messages = [preprocess(message, stem=False) for message in documents]

# bag of words model
vectorizer = CountVectorizer()
bow_model = vectorizer.fit_transform(lemma_messages)

In [48]:
# look at the dataframe
pd.DataFrame(bow_model.toarray(), columns = vectorizer.get_feature_names_out())

,account,additional,address,adnams,advertisement,advise,affect,age,ago,agoura,...,work,wtf,xfinity,yard,year,yep,youll,youre,youve,zip
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
89,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
90,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
91,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [49]:
print(vectorizer.get_feature_names_out())

['account' 'additional' 'address' 'adnams' 'advertisement' 'advise'
 'affect' 'age' 'ago' 'agoura' 'air' 'aircraft' 'alcohol' 'allow' 'almost'
 'also' 'always' 'amanda' 'amazon' 'android' 'anker' 'another' 'answer'
 'anything' 'app' 'apple' 'applesupport' 'appreciate' 'apps' 'armv'
 'article' 'ask' 'askspectrum' 'ass' 'assessment' 'assistance' 'attach'
 'avoid' 'away' 'ba' 'back' 'basket' 'battery' 'beatner' 'beg' 'behaviors'
 'believe' 'bite' 'blind' 'bluetooth' 'break' 'brilliant' 'britishairways'
 'broadside' 'browser' 'browsers' 'bug' 'business' 'buy' 'bye' 'cable'
 'cache' 'cachecookies' 'call' 'care' 'carry' 'cause' 'challenge' 'change'
 'charge' 'chat' 'check' 'cheer' 'choice' 'chris' 'click' 'close'
 'coaxial' 'code' 'colleague' 'colleagues' 'come' 'commonsense'
 'companion' 'concern' 'congrats' 'constantly' 'contact' 'cookies'
 'correct' 'could' 'couple' 'crash' 'cross' 'cumbersome' 'curiouswill'
 'currently' 'customer' 'dani' 'day' 'days' 'dead' 'delay' 'delete'
 'deliver' 'd

After applying stemming and Lemmatization  got 453 and 459 tokens out of 510 tokens

In [50]:
# bag of words model using Tfidf
 
vectorizer = TfidfVectorizer()
tfidf_model = vectorizer.fit_transform(documents)
print(tfidf_model)

  (0, 304)	0.3780748993488346
  (0, 229)	0.3780748993488346
  (0, 294)	0.3780748993488346
  (0, 434)	0.3780748993488346
  (0, 125)	0.3780748993488346
  (0, 356)	0.29243673643702095
  (0, 72)	0.3780748993488346
  (0, 27)	0.238405071414758
  (1, 358)	0.3052457302012442
  (1, 91)	0.2797276422667727
  (1, 117)	0.3052457302012442
  (1, 1)	0.3052457302012442
  (1, 84)	0.2797276422667727
  (1, 509)	0.3052457302012442
  (1, 288)	0.2361041828090945
  (1, 270)	0.14885726389373802
  (1, 121)	0.160331734632603
  (1, 323)	0.19248072335141625
  (1, 473)	0.15726110864107937
  (1, 260)	0.2797276422667727
  (1, 269)	0.2797276422667727
  (1, 58)	0.3052457302012442
  (1, 356)	0.2361041828090945
  (2, 129)	0.37447611572895667
  (2, 498)	0.37447611572895667
  :	:
  (91, 202)	0.26429752257931627
  (91, 146)	0.26429752257931627
  (91, 219)	0.26429752257931627
  (91, 354)	0.26429752257931627
  (91, 101)	0.26429752257931627
  (91, 239)	0.26429752257931627
  (91, 148)	0.26429752257931627
  (91, 48)	0.2642975225

In [51]:
# print the full sparse matrix
print(tfidf_model.toarray())

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.30524573 0.         ... 0.         0.         0.30524573]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.27726823 ... 0.         0.         0.        ]]


In [52]:
pd.DataFrame(tfidf_model.toarray(), columns = vectorizer.get_feature_names_out())

,account,additional,address,adnams,advertisement,advise,affect,age,ages,ago,...,working,wtf,xfinity,yard,year,yep,youll,youre,youve,zip
0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,0.0,0.305246,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.305246
2,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
89,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
90,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
91,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [53]:
print(tfidf_model.shape)
print(vectorizer.get_feature_names_out())

(93, 510)
['account' 'additional' 'address' 'adnams' 'advertisement' 'advise'
 'affect' 'age' 'ages' 'ago' 'agoura' 'air' 'aircraft' 'alcohol' 'allow'
 'almost' 'also' 'always' 'amanda' 'amazon' 'android' 'anker' 'another'
 'answer' 'anything' 'app' 'apple' 'applesupport' 'appreciate' 'apps'
 'armv' 'article' 'ask' 'askspectrum' 'ass' 'assessment' 'assistance'
 'attached' 'avoid' 'away' 'ba' 'back' 'basket' 'battery' 'beatner' 'beg'
 'behaviors' 'believes' 'bit' 'blind' 'bluetooth' 'brilliant'
 'britishairways' 'broadside' 'broken' 'browser' 'browsers' 'bug'
 'business' 'buying' 'bye' 'cable' 'cache' 'cachecookies' 'call' 'called'
 'calling' 'calls' 'cares' 'carry' 'cause' 'caused' 'causing' 'challenged'
 'change' 'charge' 'chat' 'check' 'cheers' 'choice' 'chris' 'click'
 'close' 'coaxial' 'code' 'colleague' 'colleagues' 'come' 'comes'
 'commonsense' 'companion' 'concern' 'congrats' 'constantly' 'contact'
 'cookies' 'correct' 'could' 'coupling' 'crash' 'crossed' 'cumbersome'
 'curiousw

In [54]:
# stem messages
stem_messages = [preprocess(message, stem=True) for message in  documents]

# bag of words model
vectorizer = TfidfVectorizer()
tfidf_model = vectorizer.fit_transform(stem_messages)
print(tfidf_model)


  (0, 271)	0.3854570571559503
  (0, 203)	0.3854570571559503
  (0, 261)	0.3854570571559503
  (0, 389)	0.3854570571559503
  (0, 110)	0.3854570571559503
  (0, 320)	0.29814675352806336
  (0, 64)	0.3303703888037585
  (0, 26)	0.24306008517587152
  (1, 321)	0.28197872663376644
  (1, 81)	0.28197872663376644
  (1, 104)	0.307702169206626
  (1, 1)	0.307702169206626
  (1, 76)	0.28197872663376644
  (1, 452)	0.307702169206626
  (1, 255)	0.2380042111030323
  (1, 239)	0.15005518004156396
  (1, 107)	0.15852665401346133
  (1, 289)	0.1940296955722981
  (1, 422)	0.15852665401346133
  (1, 230)	0.28197872663376644
  (1, 237)	0.28197872663376644
  (1, 55)	0.307702169206626
  (1, 320)	0.2380042111030323
  (2, 114)	0.3768990345608239
  (2, 442)	0.3768990345608239
  :	:
  (91, 178)	0.2713602124888724
  (91, 131)	0.2713602124888724
  (91, 193)	0.2713602124888724
  (91, 318)	0.2713602124888724
  (91, 91)	0.2713602124888724
  (91, 213)	0.2713602124888724
  (91, 46)	0.2713602124888724
  (91, 394)	0.1872088019534454

In [55]:
# look at the dataframe
pd.DataFrame(tfidf_model.toarray(), columns = vectorizer.get_feature_names_out())

,account,addit,address,adnam,advertis,advis,affect,age,ago,agoura,...,work,wtf,xfiniti,yard,year,yep,youll,your,youv,zip
0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,0.0,0.307702,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.307702
2,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
89,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
90,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
91,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [56]:
print(vectorizer.get_feature_names_out())

['account' 'addit' 'address' 'adnam' 'advertis' 'advis' 'affect' 'age'
 'ago' 'agoura' 'air' 'aircraft' 'alcohol' 'allow' 'almost' 'also' 'alway'
 'amanda' 'amazon' 'android' 'anker' 'anoth' 'answer' 'anyth' 'app' 'appl'
 'applesupport' 'appreci' 'armv' 'articl' 'ask' 'askspectrum' 'ass'
 'assess' 'assist' 'attach' 'avoid' 'away' 'ba' 'back' 'basket' 'batteri'
 'beatner' 'beg' 'behavior' 'believ' 'bit' 'blind' 'bluetooth' 'brilliant'
 'britishairway' 'broadsid' 'broken' 'browser' 'bug' 'busi' 'buy' 'bye'
 'cabl' 'cach' 'cachecooki' 'call' 'care' 'carri' 'caus' 'challeng'
 'chang' 'charg' 'chat' 'check' 'cheer' 'choic' 'chri' 'click' 'close'
 'coaxial' 'code' 'colleagu' 'come' 'commonsens' 'companion' 'concern'
 'congrat' 'constantli' 'contact' 'cooki' 'correct' 'could' 'coupl'
 'crash' 'cross' 'cumbersom' 'curiouswil' 'current' 'custom' 'dani' 'day'
 'dead' 'delay' 'delet' 'deliv' 'deliveri' 'depart' 'departur' 'detail'
 'devic' 'differ' 'direct' 'disconnect' 'disgrac' 'disregard' 'dis

In [57]:
# lemmatise messages
lemma_messages = [preprocess(message, stem=False) for message in documents]

# bag of words model
vectorizer = TfidfVectorizer()
tfidf_model = vectorizer.fit_transform(lemma_messages)

In [58]:
# look at the dataframe
pd.DataFrame(tfidf_model.toarray(), columns = vectorizer.get_feature_names_out())

,account,additional,address,adnams,advertisement,advise,affect,age,ago,agoura,...,work,wtf,xfinity,yard,year,yep,youll,youre,youve,zip
0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,0.0,0.307702,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.307702
2,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
89,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
90,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
91,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [59]:
print(tfidf_model.shape)
print(vectorizer.get_feature_names_out())

(93, 459)
['account' 'additional' 'address' 'adnams' 'advertisement' 'advise'
 'affect' 'age' 'ago' 'agoura' 'air' 'aircraft' 'alcohol' 'allow' 'almost'
 'also' 'always' 'amanda' 'amazon' 'android' 'anker' 'another' 'answer'
 'anything' 'app' 'apple' 'applesupport' 'appreciate' 'apps' 'armv'
 'article' 'ask' 'askspectrum' 'ass' 'assessment' 'assistance' 'attach'
 'avoid' 'away' 'ba' 'back' 'basket' 'battery' 'beatner' 'beg' 'behaviors'
 'believe' 'bite' 'blind' 'bluetooth' 'break' 'brilliant' 'britishairways'
 'broadside' 'browser' 'browsers' 'bug' 'business' 'buy' 'bye' 'cable'
 'cache' 'cachecookies' 'call' 'care' 'carry' 'cause' 'challenge' 'change'
 'charge' 'chat' 'check' 'cheer' 'choice' 'chris' 'click' 'close'
 'coaxial' 'code' 'colleague' 'colleagues' 'come' 'commonsense'
 'companion' 'concern' 'congrats' 'constantly' 'contact' 'cookies'
 'correct' 'could' 'couple' 'crash' 'cross' 'cumbersome' 'curiouswill'
 'currently' 'customer' 'dani' 'day' 'days' 'dead' 'delay' 'delete'
 'd

In [60]:
vectorizer.vocabulary_

{'applesupport': 26,
 'cause': 66,
 'reply': 325,
 'disregard': 115,
 'tap': 393,
 'notification': 266,
 'keyboard': 207,
 'open': 276,
 'business': 57,
 'mean': 241,
 'lot': 234,
 'us': 425,
 'please': 294,
 'direct': 112,
 'message': 243,
 'name': 260,
 'zip': 458,
 'code': 78,
 'additional': 1,
 'detail': 108,
 'concern': 84,
 'request': 326,
 'really': 314,
 'hope': 176,
 'change': 68,
 'im': 185,
 'sure': 385,
 'wont': 448,
 'dont': 119,
 'livechat': 227,
 'online': 274,
 'moment': 255,
 'contact': 87,
 'option': 278,
 'leave': 218,
 'call': 63,
 'back': 40,
 'virgintrains': 429,
 'see': 336,
 'attach': 36,
 'error': 128,
 'ive': 198,
 'try': 414,
 'voicemail': 431,
 'several': 342,
 'time': 409,
 'past': 287,
 'week': 437,
 'another': 21,
 'device': 109,
 'miriam': 250,
 'yep': 454,
 'laptop': 212,
 'today': 410,
 'different': 111,
 'browsers': 55,
 'work': 449,
 'ok': 270,
 'link': 224,
 'help': 170,
 'still': 377,
 'havent': 167,
 'hear': 168,
 'number': 268,
 'phone': 291,
 'd

 N-grams

In [61]:

bigram=CountVectorizer(ngram_range=(2,2))

bigramvocab=bigram.fit_transform(documents)


len(bigram.vocabulary_)
bigram.vocabulary_


{'applesupport causing': 31,
 'causing reply': 97,
 'reply disregarded': 514,
 'disregarded tapped': 155,
 'tapped notification': 627,
 'notification keyboard': 430,
 'keyboard opened': 333,
 'business means': 82,
 'means lot': 391,
 'lot us': 379,
 'us please': 707,
 'please direct': 472,
 'direct message': 151,
 'message name': 397,
 'name zip': 424,
 'zip code': 774,
 'code additional': 106,
 'additional details': 3,
 'details concern': 141,
 'concern reply': 116,
 'reply requested': 515,
 'really hope': 501,
 'hope change': 278,
 'change im': 99,
 'im sure': 293,
 'sure wont': 617,
 'wont dont': 758,
 'livechat online': 363,
 'online moment': 443,
 'moment contact': 414,
 'contact option': 119,
 'option leave': 448,
 'leave message': 349,
 'message request': 398,
 'request call': 516,
 'call back': 88,
 'virgintrains see': 730,
 'see attached': 530,
 'attached error': 54,
 'error message': 174,
 'message ive': 395,
 'ive tried': 325,
 'tried leaving': 676,
 'leaving voicemail': 350

In [62]:
bigram=CountVectorizer(ngram_range=(3,3))

bigramvocab=bigram.fit_transform(documents)


bigram.vocabulary_

{'applesupport causing reply': 29,
 'causing reply disregarded': 90,
 'reply disregarded tapped': 494,
 'disregarded tapped notification': 152,
 'tapped notification keyboard': 594,
 'notification keyboard opened': 416,
 'business means lot': 78,
 'means lot us': 374,
 'lot us please': 364,
 'us please direct': 670,
 'please direct message': 456,
 'direct message name': 146,
 'message name zip': 381,
 'name zip code': 410,
 'zip code additional': 733,
 'code additional details': 99,
 'additional details concern': 2,
 'details concern reply': 133,
 'concern reply requested': 108,
 'really hope change': 484,
 'hope change im': 265,
 'change im sure': 92,
 'im sure wont': 279,
 'sure wont dont': 584,
 'livechat online moment': 349,
 'online moment contact': 428,
 'moment contact option': 400,
 'contact option leave': 111,
 'option leave message': 433,
 'leave message request': 333,
 'message request call': 382,
 'request call back': 495,
 'virgintrains see attached': 691,
 'see attached e

In [63]:
bigram=CountVectorizer(ngram_range=(4,4))

bigramvocab=bigram.fit_transform(documents)


bigram.vocabulary_

{'applesupport causing reply disregarded': 29,
 'causing reply disregarded tapped': 86,
 'reply disregarded tapped notification': 453,
 'disregarded tapped notification keyboard': 148,
 'tapped notification keyboard opened': 538,
 'business means lot us': 75,
 'means lot us please': 344,
 'lot us please direct': 336,
 'us please direct message': 604,
 'please direct message name': 420,
 'direct message name zip': 139,
 'message name zip code': 350,
 'name zip code additional': 377,
 'zip code additional details': 663,
 'code additional details concern': 94,
 'additional details concern reply': 2,
 'details concern reply requested': 126,
 'really hope change im': 445,
 'hope change im sure': 244,
 'change im sure wont': 88,
 'im sure wont dont': 256,
 'livechat online moment contact': 324,
 'online moment contact option': 394,
 'moment contact option leave': 367,
 'contact option leave message': 105,
 'option leave message request': 399,
 'leave message request call': 305,
 'message req

In [64]:
len(bigram.vocabulary_)


664

OneHotEncoder

In [65]:
values = np.array(documents)
print(values)

['applesupport causing reply disregarded tapped notification keyboard opened'
 'business means lot us please direct message name zip code additional details concern reply requested'
 'really hope change im sure wont dont'
 'livechat online moment contact option leave message request call back'
 'virgintrains see attached error message ive tried leaving voicemail several times past week'
 'tried another device miriam message'
 'virgintrains yep ive tried laptop several times past week today ive tried different browsers'
 'working ok miriam link help message'
 'virgintrains still havent heard number im directed phone dead end live chat doesnt work someone call'
 'thats miriam team send email shortly happy'
 'help version ios find settings general reply direct message'
 'applesupport suffering hope find solution'
 'applesupport hi ’ concern latest ios slow happy solution please'
 'thanks reaching us always happy help send us direct message look together'
 'updated phone suddenly everythin

In [66]:
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = values.reshape(len(values), 1)
integer_encoded

array([['applesupport causing reply disregarded tapped notification keyboard opened'],
       ['business means lot us please direct message name zip code additional details concern reply requested'],
       ['really hope change im sure wont dont'],
       ['livechat online moment contact option leave message request call back'],
       ['virgintrains see attached error message ive tried leaving voicemail several times past week'],
       ['tried another device miriam message'],
       ['virgintrains yep ive tried laptop several times past week today ive tried different browsers'],
       ['working ok miriam link help message'],
       ['virgintrains still havent heard number im directed phone dead end live chat doesnt work someone call'],
       ['thats miriam team send email shortly happy'],
       ['help version ios find settings general reply direct message'],
       ['applesupport suffering hope find solution'],
       ['applesupport hi ’ concern latest ios slow happy solution plea

In [67]:
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)

[[1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [68]:
(onehot_encoded).shape

(93, 91)

In [69]:
#!Pip install textblob
#from textblob import TextBlob
#res = TextBlob("The weather today is pleasant.")
#print(res.sentiment.polarity)


One-Hot Encoding (OHE):

Advantages:

1.	Simple and easy to implement.
2.	Preserves word order.
3.	Suitable for small vocabularies.
4.	Intuitive representation for classification tasks.
5.	Can handle words not seen during training.



Disadvantages:

1.	High dimensionality, especially for large vocabularies.
2.	Does not capture semantic relationships between words.
3.	Sparse representation, leading to memory and computational inefficiency.
4.	Cannot handle out-of-vocabulary words without additional handling.
5.	May result in biased models due to the imbalance of word frequencies.


Bag of Words (BOW):

Advantages:

1.	Simple and efficient representation.
2.	Preserves word frequency information.
3.	Captures document structure effectively.
4.	Suitable for large datasets and online learning.
5.	Can be easily combined with other feature extraction techniques.


Disadvantages:

1.	Ignores word order and context.
2.	Loss of semantic meaning and relationships between words.
3.	High dimensionality, especially for large vocabularies.
4.	Sparse representation, leading to memory and computational inefficiency.
5.	Inability to handle out-of-vocabulary words without additional handling.



N-grams:

Advantages:

1.	Captures local word dependencies and context.
2.	Preserves some sequential information compared to BOW.
3.	Can improve model performance for tasks like sentiment analysis and text classification.
4.	Provides flexibility in choosing the size of n-grams.
5.	Can handle out-of-vocabulary words better than BOW.


Disadvantages:

1.	Increased dimensionality, especially for higher values of n.
2.	Greater memory and computational requirements compared to BOW.
3.	Limited ability to capture long-range dependencies between words.
4.	Vulnerable to noise and sparsity for large n-gram sizes.
5.	Limited generalization to unseen n-grams, especially for smaller datasets.


TF-IDF (Term Frequency-Inverse Document Frequency):

Advantages:

1.	Captures word importance within documents and across the corpus.
2.	Penalizes common words while highlighting rare words.
3.	Reduces the impact of stop words and noise in the data.
4.	Provides a dense representation compared to BOW and OHE.
5.	Effective for information retrieval tasks like document ranking and recommendation systems.



Disadvantages:

1.	Ignores word order and context like BOW.
2.	Requires tuning of hyperparameters, such as the smoothing parameter.
3.	Can be sensitive to document length and corpus size.
4.	Does not capture semantic relationships between words.
5.	Complexity in interpretation compared to BOW and OHE.
